In [ ]:
import sys
from datasets import load_dataset
import torch
from trl.trainer.dpo_trainer import DPOTrainer

sys.path.append('..')
from model.causal_lm import AutoLMFineTuner
from model.utils import LMHyperParams, SmModel, ModelChoice
from train import start_training
from dataset.squad import UltraFeedbackDataModule
%load_ext autoreload
%autoreload 2

2024-11-14 00:29:41.447 | INFO     | train:<module>:5 - Loading dependencies - Torch...


2024-11-14 00:29:41.557 | INFO     | train:<module>:16 - Loading dependencies - Lightning...
2024-11-14 00:29:41.558 | INFO     | train:<module>:25 - Loading dependencies - Project...


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [25]:
params = LMHyperParams(base_model_checkpoint="Qwen/Qwen2.5-0.5B", warmup_steps_count=10, tuning_type="dpo")

# https://github.com/philschmid/deep-learning-pytorch-huggingface/blob/main/training/dpo-align-llms-in-2024-with-trl.ipynb

In [26]:
model = AutoLMFineTuner(params)


In [35]:
data_module = UltraFeedbackDataModule(16, model.tokenizer, 1024)
data_module.setup("fit")

2024-11-14 03:10:44.481 | INFO     | model.utils:setup:90 - Loading dataset for stage fit


2024-11-14 03:10:45.596 | INFO     | model.utils:setup:98 - Processing dataset for stage fit, workers: 1


In [31]:
start_training(
    "16-mixed", "", model, model.tokenizer, data_module, params, False, ""
)


Seed set to 42
2024-11-14 03:10:15.425 | INFO     | train:start_training:268 - Effective batch size: 8 (2x4)
/admin/home-brianf/smolmodels/.venv/lib/python3.11/site-packages/lightning/fabric/plugins/environments/slurm.py:204: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python /admin/home-brianf/smolmodels/.venv/lib/python3.11/s ...
Using 16bit Automatic Mixed Precision (AMP)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
2024-11-14 03:10:15.491 | INFO     | model.utils:setup:90 - Loading dataset for stage TrainerFn.FITTING

Detected KeyboardInterrupt, attempting graceful shutdown ...


NameError: name 'exit' is not defined